# Проект: Анализ лиц на фотографиях из VK-альбома
**Альбом ВКонтакте:** https://vk.com/album-130344439_305487633

## Описание проекта
В этом проекте мы анализируем фотографии из открытых альбомов VK, извлекаем лица, получаем эмбеддинги, строим граф встреч между персонами и визуализируем результаты. 

## Описание датасета
- **Источник:** фотографии из альбома VK `https://vk.com/album-130344439_305487633`
- **photo_id** — ID фото в VK
- **face_id** — уникальный ID лица
- **embedding** — эмбеддинг (512-мерный вектор)
- **bbox** — координаты лица (x, y, width, height)
- **album_id** — ID альбома
- **source_url** — ссылка на фото

## Импорт библиотек

In [1]:
import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd
import face_recognition
import networkx as nx
from itertools import combinations
from pyvis.network import Network
import streamlit as st
import matplotlib.pyplot as plt
import streamlit.components.v1 as components
import os
import requests
import ast

/Users/anastasiafirsova/photo_env/lib/python3.13/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## Скачивание изображений из VK
Мы используем VK API, чтобы получить фотографии из открытого альбома.

In [2]:
access_token = "vk1.a.JNcse_34a53EI22HdfTWl2OKeJ57xqXwxMLzIuoB7tftRyd54Zr5ybvKBec3tWIJTkaqUYxno9RSEk86QDaNtxAj0ga9eASo85oyAK2FWW1HsXQaOfnnBj7bvGdP0PM53tdEIfzj_sMVIrT0fA946q2mW1IkDfDUCvZcZ26Anwb3R3aCC_UpRTCRkNrUmvutLBeYq_HMrVAjKYKazPiCbg"
api_version = "5.131"
owner_id = "-130344439"     
album_id = "305487633"      # id альбома VK

# Получаем список фотографий в альбоме
photos_url = (
    f"https://api.vk.com/method/photos.get?"
    f"owner_id={owner_id}&album_id={album_id}"
    f"&access_token={access_token}&v={api_version}&count=1000"
)
res = requests.get(photos_url).json()
items = res.get("response", {}).get("items", [])
os.makedirs("images", exist_ok=True)
for photo in items:
    # Берём максимальный размер (последний в списке sizes)
    photo_url = photo["sizes"][-1]["url"]
    photo_id = photo["id"]
    filename = f"images/photo_{photo_id}.jpg"
    # Скачиваем и сохраняем файл
    img_data = requests.get(photo_url).content
    with open(filename, "wb") as f:
        f.write(img_data)
    print(f"Saved {filename}")

Saved images/photo_457248653.jpg
Saved images/photo_457248654.jpg
Saved images/photo_457248655.jpg
Saved images/photo_457248656.jpg
Saved images/photo_457248657.jpg
Saved images/photo_457248658.jpg
Saved images/photo_457248659.jpg
Saved images/photo_457248660.jpg
Saved images/photo_457248661.jpg
Saved images/photo_457248662.jpg
Saved images/photo_457248663.jpg
Saved images/photo_457248664.jpg
Saved images/photo_457248665.jpg
Saved images/photo_457248666.jpg
Saved images/photo_457248667.jpg
Saved images/photo_457248668.jpg
Saved images/photo_457248669.jpg
Saved images/photo_457248670.jpg
Saved images/photo_457248671.jpg
Saved images/photo_457248672.jpg
Saved images/photo_457248673.jpg
Saved images/photo_457248674.jpg
Saved images/photo_457248675.jpg
Saved images/photo_457248676.jpg
Saved images/photo_457248677.jpg
Saved images/photo_457248678.jpg
Saved images/photo_457248679.jpg
Saved images/photo_457248680.jpg
Saved images/photo_457248681.jpg
Saved images/photo_457248682.jpg
Saved imag

## Распознавание лиц

In [ ]:
data = []
face_id_counter = 0

for image_file in os.listdir("images"):
    image = face_recognition.load_image_file(f"images/{image_file}")
    # Находим все лица: возвращается список bbox (top, right, bottom, left)
    face_locations = face_recognition.face_locations(image)
    # Получаем 128-мерные эмбеддинги для найденных лиц
    face_encodings = face_recognition.face_encodings(image, face_locations)
    photo_id = int(image_file.split("_")[1].split(".")[0])  # извлечение ID из имени
    
    for loc, emb in zip(face_locations, face_encodings):
        (top, right, bottom, left) = loc
        face_id_counter += 1
        data.append({
            "photo_id": photo_id,
            "face_id": face_id_counter,
            "embedding": emb.tolist(),
            "bbox": (left, top, right-left, bottom-top),
            "album_id": album_id,
            "source_url": np.nan  # сделаем это позже
        })

# Сохраняем результат в CSV
df = pd.DataFrame(data)
df.to_csv("faces.csv", index=False)
print(f"Обнаружено {len(df)} лиц на {len(items)} фотографиях")

## Кластеризация лиц и генерация person_id

In [ ]:
encodings = np.vstack(df["embedding"].to_numpy())  # матрица N×128
clt = DBSCAN(metric="euclidean", eps=0.42, min_samples=2).fit(encodings)
labels = clt.labels_  # label - идентификатор кластера (человека)
df["person_id"] = labels
num_persons = len(set(labels))
print(f"Найдено {num_persons} уникальных людей")

In [ ]:
group_id = "-130344439"
photo_url_shab = "https://vk.com/photo{}_{}"  # group_id, photo_id

# Добавляем album_id
df["album_id"] = album_id

# Формируем source_url
df["source_url"] = df["photo_id"].apply(
    lambda pid: photo_url_shab.format(group_id, pid)
)

## Данные

In [ ]:
df.head()

#### При использовании данного способа появляется person_id = -1: 
#### Это плохо распознанные, размытые лица (например, фото с самим посвящением, где  лица прикрыты), эти объекты считаются шумом и не относятся ни к одному кластеру

In [ ]:
df_clean = df[df["person_id"] != -1].copy()
df = df_clean.copy()

## Построение графа по co-occurrence

In [ ]:
G = nx.Graph()
# Добавляем узлы (по person_id)
G.add_nodes_from(labels)
# Для каждой фотографии: получаем list уникальных person_id
for pid in df["photo_id"].unique():
    persons = set(df[df["photo_id"] == pid]["person_id"])
    for u, v in combinations(persons, 2):
        if G.has_edge(u, v):
            G[u][v]["weight"] += 1
        else:
            G.add_edge(u, v, weight=1)

# Сохраняем ребра графа в таблицу
edges = [(u, v, G[u][v]["weight"]) for u, v in G.edges()]
edges_df = pd.DataFrame(edges, columns=["person_id_1", "person_id_2", "cooccurrence_count"])
edges_df.to_csv("graph_edges.csv", index=False)
graph_edges = pd.read_csv('graph_edges.csv')
graph_edges

In [ ]:
print("Всего обнаруженных лиц:", len(df))
print("Всего обнаруженных людей:", df['person_id'].nunique())
print("Топ-5 активных пользователей:")
print(df["person_id"].value_counts().head(5))

## Визуализация графа (NetworkX)

In [ ]:
edges_graph_edgesf = pd.read_csv("graph_edges.csv")

G = nx.Graph()
for _, row in graph_edges.iterrows():
    G.add_edge(row["person_id_1"], row["person_id_2"], weight=row["cooccurrence_count"])

node_size = [100 + 50 * G.degree(n) for n in G.nodes()]
edge_width = [G[u][v]['weight'] for u, v in G.edges()]
pos = nx.spring_layout(G, k=0.5, iterations=50)

plt.figure(figsize=(12, 12))
nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='skyblue', alpha=0.8)
nx.draw_networkx_edges(G, pos, width=edge_width, edge_color='gray', alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=8)

plt.title("Социальный граф: встречи по фотографиям")
plt.axis("off")
plt.tight_layout()

## Визуализация HTML

In [ ]:
G = nx.Graph()
for _, row in edges_df.iterrows():
    u = str(int(row["person_id_1"]))
    v = str(int(row["person_id_2"]))
    weight = int(row["cooccurrence_count"]) 
    G.add_edge(u, v, weight=weight)

In [ ]:
net = Network(height='600px', width='100%', notebook=True)

In [ ]:
net = Network(height='600px', width='100%', notebook=True)
net.from_nx(G)
net.show("vk_faces.html") 